In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/캡스톤/weather.csv")

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

,Date,T2M,WS2M,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_UV_INDEX,PRECTOTCORR,RH2M,PS,locationName,season_tag,days_until_harvest
0,2015-01-01,25.08,1.81,8.12,3.47,0.19,75.00,91.62,brazil_varginha,off-season,120
1,2015-01-02,25.16,1.84,7.59,3.23,0.30,78.66,91.60,brazil_varginha,off-season,119
2,2015-01-03,25.86,1.01,6.72,2.87,7.44,72.98,91.67,brazil_varginha,off-season,118
3,2015-01-04,23.68,1.58,6.38,2.73,1.79,80.20,91.80,brazil_varginha,off-season,117
4,2015-01-05,22.72,1.99,5.83,2.44,4.99,82.20,91.85,brazil_varginha,off-season,116


In [ ]:
df = df.drop(columns=['T2M','WS2M',	'ALLSKY_SFC_SW_DWN',	'ALLSKY_SFC_UV_INDEX','RH2M',	'PS', 'days_until_harvest'])

In [ ]:
df_melted = df.melt(id_vars=["Date", "locationName"], var_name="Indicator", value_name="Value")
df_transformed = df_melted.pivot(index="Date", columns=["locationName", "Indicator"], values="Value").reset_index()

# MultiIndex 컬럼을 단일 컬럼명("지명_지표")으로 변환
df_transformed.columns = ["Date"] + [f"{country}_{indicator}" for country, indicator in df_transformed.columns[1:]]

기후데이터의 결측치 처리

In [ ]:
df_transformed.fillna(method='ffill', inplace=True)  # 이전 값으로 대체
df_transformed.fillna(method='bfill', inplace=True)  # 다음 값으로 대체

<ipython-input-28-ae7ae87fef9a>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_transformed.fillna(method='ffill', inplace=True)  # 이전 값으로 대체
<ipython-input-28-ae7ae87fef9a>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_transformed.fillna(method='ffill', inplace=True)  # 이전 값으로 대체
<ipython-input-28-ae7ae87fef9a>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_transformed.fillna(method='bfill', inplace=True)  # 다음 값으로 대체


In [ ]:
df_transformed.isna().sum().sum()

np.int64(0)

In [ ]:
df

,Date,PRECTOTCORR,locationName,season_tag
0,2015-01-01,0.19,brazil_varginha,off-season
1,2015-01-02,0.30,brazil_varginha,off-season
2,2015-01-03,7.44,brazil_varginha,off-season
3,2015-01-04,1.79,brazil_varginha,off-season
4,2015-01-05,4.99,brazil_varginha,off-season
...,...,...,...,...
33597,2025-03-19,10.25,colombia_pereira,pre-harvest
33598,2025-03-20,7.96,colombia_pereira,pre-harvest
33599,2025-03-21,12.10,colombia_pereira,pre-harvest
33600,2025-03-22,12.16,colombia_pereira,pre-harvest


가장 최근 수확기의 평균 값이 매핑될 수 있도록 함

In [ ]:
weather_vars = [col for col in df_transformed.columns if col not in ['Date'] and not col.endswith('season_tag')]

In [ ]:
season_tag_cols = [col for col in df_transformed.columns if col.endswith('season_tag')]

In [ ]:
def find_non_harvest_blocks(series: pd.Series):
    blocks = []
    in_block = False
    start = None

    for i in range(len(series)):
        if series.iloc[i] != 'harvest':  # 'non-harvest' 인 경우
            if not in_block:
                start = series.index[i]
                in_block = True
        else:
            if in_block:
                end = series.index[i - 1]
                blocks.append((start, end))
                in_block = False

    if in_block:
        end = series.index[-1]
        blocks.append((start, end))
    return blocks

In [ ]:
non_harvest_info = {}

for season_col in season_tag_cols:
    region = season_col.replace('_season_tag', '')
    region_weather_cols = [col for col in weather_vars if col.startswith(region)]

    tag_series = df_transformed.set_index('Date')[season_col]
    blocks = find_non_harvest_blocks(tag_series)

    for i, (start_date, end_date) in enumerate(blocks):
        block_mask = (df_transformed['Date'] >= start_date) & (df_transformed['Date'] <= end_date)
        block_df_transformed = df_transformed[block_mask]

        for col in region_weather_cols:
            mean_val = block_df_transformed[col].mean()
            key = f"{col}_prev_nonharvest_mean_{i+1}"
            non_harvest_info[key] = {
                'mean': mean_val,
                'region': region,
                'variable': col.split(f"{region}_")[1],
                'start_date': start_date,
                'end_date': end_date,
                'original_col': col
            }

In [ ]:
from datetime import datetime

def group_harvest_info_by_col(harvest_info: dict):
    grouped = {}
    for key, info in harvest_info.items():
        col = info['original_col']
        if col not in grouped:
            grouped[col] = []
        grouped[col].append(info)

    for col in grouped:
        grouped[col] = sorted(grouped[col], key=lambda x: pd.to_datetime(x['end_date']))
    return grouped

def apply_recent_harvest_mean(df, grouped_info):
    for original_col, blocks in grouped_info.items():
        feature_col = f"{original_col}_harvest_mean"

        def get_recent_mean(current_date):
            current_date = pd.to_datetime(current_date)
            for block in reversed(blocks):  # 가장 최근부터 검사
                if current_date > pd.to_datetime(block['end_date']):
                    return block['mean']
            return None  # 해당 없음 (모든 end_date 이후가 아님)

        df[feature_col] = df['Date'].apply(get_recent_mean)

    return df

In [ ]:
grouped_nonharvest = group_harvest_info_by_col(non_harvest_info)
df_new = apply_recent_harvest_mean(df_transformed, grouped_nonharvest)

In [ ]:
cols = ['Date'] + [col for col in df_new.columns if col.endswith('_mean')]

In [ ]:
df_final = df_new[cols]

In [ ]:
df_final = df_final.dropna()

In [ ]:
len(df_final)

3462

커피 가격 데이터와 merge

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/캡스톤/커피가격데이터통합.csv")

In [ ]:
data.fillna(method='ffill', inplace=True)  # 이전 값으로 대체
data.fillna(method='bfill', inplace=True)  # 다음 값으로 대체

<ipython-input-41-6fe7a7a35e26>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)  # 이전 값으로 대체
<ipython-input-41-6fe7a7a35e26>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill', inplace=True)  # 다음 값으로 대체


In [ ]:
final = pd.merge(data, df_final, on='Date', how='left')

In [ ]:
final = final.dropna()

In [ ]:
final.to_csv('비수확기평균커피가격통합데이터.csv', index=False)